# Docker-compose manipulation

In [19]:
#Build images
!docker-compose build --no-cache


traefik uses an image, skipping
shard2_2 uses an image, skipping
shard1_3_userDB uses an image, skipping
shard1_3 uses an image, skipping
shard1_2_userDB uses an image, skipping
shard1_2 uses an image, skipping
clickhouse uses an image, skipping
mongodb-exporter-user-db uses an image, skipping
shard2_3 uses an image, skipping
shard2_1 uses an image, skipping
shard2_3_userDB uses an image, skipping
postgres-exporter uses an image, skipping
mongodb-exporter-manager-db uses an image, skipping
shard1_1_userDB uses an image, skipping
redis-exporter uses an image, skipping
shard2_2_userDB uses an image, skipping
shard2_1_userDB uses an image, skipping
configsvr1_userDB uses an image, skipping
configsvr2_userDB uses an image, skipping
configsvr3_userDB uses an image, skipping
shard1_1 uses an image, skipping
configsvr1 uses an image, skipping
configsvr2 uses an image, skipping
configsvr3 uses an image, skipping
mongo-express uses an image, skipping
prometheus uses an image, skipping
mongo-exp

In [20]:
#Run containers
!docker-compose up -d

Creating network "parkman_app-network" with driver "bridge"
Creating parkman_shard1_2_userDB_1 ... 
Creating mongodb-exporter-user-db  ... 
Creating parkman_park_transactions_db_1 ... 
Creating parkman_shard2_2_userDB_1      ... 
Creating parkman_shard1_3_userDB_1      ... 
Creating postgres-exporter              ... 
Creating parkman_shard2_2_1             ... 
Creating parkman_shard2_3_userDB_1      ... 
Creating redis_exporter                 ... 
Creating mongodb-exporter-manager-db    ... 
Creating parkman_shard2_3_1             ... 
Creating parkman_shard1_3_1             ... 
Creating parkman_redis_parking_spots_status_1 ... 
Creating parkman_traefik_1                    ... 
Creating parkman_shard1_2_1                   ... 
Creating clickhouse                           ... 
ing parkman_shard1_3_userDB_1            ... doneCreating parkman_shard1_1_userDB_1            ... 
Creating parkman_shard2_1_userDB_1            ... mdone
ting parkman_traefik_1                    ... done

In [18]:
#Stop containers
!docker-compose  down

Stopping nginx_exporter                       ... 
Stopping parkman_nginx_1                      ... 
Stopping mobile_app                           ... 
Stopping manager_app                          ... 
Stopping sensor_app                           ... 
Stopping mexpress                             ... 
Stopping etl_container                        ... 
Stopping mexpress_userDB                      ... 
Stopping mongo_user_db_service                ... 
Stopping mongo_manager_db_service             ... 
Stopping parkman_configsvr3_userDB_1          ... 
Stopping parkman_configsvr3_1                 ... 
Stopping parkman_configsvr2_userDB_1          ... 
Stopping parkman_configsvr2_1                 ... 
Stopping parkman_configsvr1_1                 ... 
Stopping parkman_configsvr1_userDB_1          ... 
Stopping parkman_shard1_1_1                   ... 
Stopping parkman_shard1_1_userDB_1            ... 
Stopping parkman_shard2_1_1                   ... 
Stopping parkman_shard2_1_userD

# Main setup notebook for docker compose
This segment exposes cells for defining docker-compose structure

## Docker-compose config

The cell below creates an empty docker-compose file. And clears out the .env of docker compose. <span style="color:red">**WARNING**</span> - running this cell will overwrite current docker compose. It should be run first when beginning config.

In [24]:
%%writefile docker-compose.yml
version: '3'

services:



networks:
  app-network:
    driver: bridge  # Use the default bridge network driver

volumes:

Overwriting docker-compose.yml


In [ ]:
%% writefile .env

In [ ]:
with open("./.env", "a") as f:
    f.write ("\nCOMPOSE_PATH_SEPARATOR=:")
    f.write("\nCOMPOSE_FILE=redundancy-manager-db.yml:redundancy-user-db.yml:clickhouse.yml:python-etl.yml:prometheus_service.yml:docker-compose.yml")

In [ ]:
%% writefile ./ManagerApp/.env

In [ ]:
%% writefile ./MobileApp/.env

## Creating ManagerApp and MobileApp services

Define requirements required for source code to successfully work in container

### ManagerAPP section

In [56]:
%%writefile ./ManagerApp/requirements.txt

Flask
requests
python-dotenv
pymongo



Overwriting ./ManagerApp/requirements.txt


In [26]:
%%writefile ./ManagerApp/app.dockerfile

FROM python:3.12

# Set the working directory inside the container to /app
WORKDIR /app

# Copy the requirements.txt file into the container at /app
COPY requirements.txt .

# Install the Python dependencies specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Copy the application code into the container at /app
COPY ./source_code/ ./source_code/

# Expose port 80 to allow external access to the container's port 80
EXPOSE 80

# Specify the command to run when the container starts
CMD ["python", "./source_code/API_voditelj/app.py"]

Overwriting ./ManagerApp/app.dockerfile


The cell below adds necessary config to define ManagerApp service 

In [27]:
 
from ruamel.yaml import YAML

# Create a YAML object
yaml = YAML()
yaml.preserve_quotes = True  # Preserves quotes in the YAML file
yaml.indent(mapping = 2, sequence = 2, offset = 2)

#Setup file edit path 
docker_compose_path = './docker-compose.yml'  

# Read the docker-compose.yml file
with open(docker_compose_path, 'r') as file:
    docker_compose = yaml.load(file)

if docker_compose['services'] is None:
    docker_compose['services']={}

if docker_compose['volumes'] is None:
    docker_compose['volumes']={}

ManagerApp = {
    'container_name': 'manager_app', #TODO: important for nginx detection
    'build': {
        'context': './ManagerApp',
        'dockerfile': 'app.dockerfile'
    },
    'networks': [
        'app-network'
    ],
    'depends_on':{},
    'env_file':[]



}

#Add the ManagerApp service to the docker-compose.yml file
docker_compose['services']['manager_app'] = ManagerApp

# Write the updated configuration back to docker-compose.yml
with open(docker_compose_path, 'w') as file:
    yaml.dump(docker_compose, file)

print("\ndocker-compose.yml has been updated successfully.")
 


docker-compose.yml has been updated successfully.


### MobileApp section

Create requirements for python to exec code properly

In [57]:
%%writefile ./MobileApp/requirements.txt

Flask
psycopg2
python-dotenv
requests
pymongo



Overwriting ./MobileApp/requirements.txt


Generate app dockerfile for MobileAPP

In [2]:
%%writefile ./MobileApp/app.dockerfile

FROM python:3.12

# Set the working directory inside the container to /app
WORKDIR /app

# Copy the requirements.txt file into the container at /app
COPY requirements.txt .

# Install the Python dependencies specified in requirements.txt
RUN pip install --no-cache-dir -r requirements.txt

# Copy the application code into the container at /app
COPY ./source_code/ ./source_code/

# Expose port 80 to allow external access to the container's port 80
EXPOSE 80

# Specify the command to run when the container starts
CMD ["python", "./source_code/API_korisnik/app.py"]

Writing ./MobileApp/app.dockerfile


Modify docker compose so that it adds second part of app as service

In [4]:
 
from ruamel.yaml import YAML

# Create a YAML object
yaml = YAML()
yaml.preserve_quotes = True  # Preserves quotes in the YAML file
yaml.indent(mapping = 2, sequence = 2, offset = 2)

#Setup file edit path 
docker_compose_path = './docker-compose.yml'  

# Read the docker-compose.yml file
with open(docker_compose_path, 'r') as file:
    docker_compose = yaml.load(file)

if docker_compose['services'] is None:
    docker_compose['services']={}

if docker_compose['volumes'] is None:
    docker_compose['volumes']={}

MobileAPP = {
    'container_name': 'mobile_app', #TODO: important for nginx detection
    'build': {
        'context': './MobileApp',
        'dockerfile': 'app.dockerfile'
    },
    'networks': [
        'app-network'
    ],
    'depends_on':{},
    'env_file':[]



}

#Add the ManagerApp service to the docker-compose.yml file
docker_compose['services']['mobile_app'] = MobileAPP

# Write the updated configuration back to docker-compose.yml
with open(docker_compose_path, 'w') as file:
    yaml.dump(docker_compose, file)

print("\ndocker-compose.yml has been updated successfully.")
 


docker-compose.yml has been updated successfully.


## Sensors section

In [ ]:
%%writefile ./Sensors/requirements.txt

Flask
redis
python-dotenv
pymongo



Overwriting ./MobileApp/requirements.txt


In [ ]:
%%writefile ./Sensors/app.dockerfile


FROM python:3.12

#Set working directory in the container
WORKDIR /app

#Copy requirements.txt
COPY requirements.txt .

#Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

#Copy the application code into the container at /app
COPY ./source_code/ ./source_code/

#Expose port 80 inside the container
EXPOSE 80

#Run the app
CMD ["python", "./source_code/API_senzori/app.py"]

Writing ./MobileApp/app.dockerfile


In [ ]:
 
from ruamel.yaml import YAML

# Create a YAML object
yaml = YAML()
yaml.preserve_quotes = True  # Preserves quotes in the YAML file
yaml.indent(mapping = 2, sequence = 2, offset = 2)

#Setup file edit path 
docker_compose_path = './docker-compose.yml'  

# Read the docker-compose.yml file
with open(docker_compose_path, 'r') as file:
    docker_compose = yaml.load(file)

if docker_compose['services'] is None:
    docker_compose['services']={}

if docker_compose['volumes'] is None:
    docker_compose['volumes']={}

sensors = {
    'container_name': 'sensor_app', #TODO: important for nginx detection
    'build': {
        'context': './Sensors',
        'dockerfile': 'app.dockerfile'
    },
    'networks': [
        'app-network'
    ],
    'depends_on':{
        'redis_parking_spots_status':{'condition':'service_healthy'}
    },
    'env_file':['./Sensors/.env']



}

#Add the ManagerApp service to the docker-compose.yml file
docker_compose['services']['sensor_app'] = sensors

# Write the updated configuration back to docker-compose.yml
with open(docker_compose_path, 'w') as file:
    yaml.dump(docker_compose, file)

print("\ndocker-compose.yml has been updated successfully.")
 


docker-compose.yml has been updated successfully.


# Other Docker commands

In [ ]:
#Purge all docker images
!docker system purge -a -f

In [1]:
#Purge all docker volumes
!docker volume prune -f

Deleted Volumes:
5b40b49505ca585a6cb559840c5804eb4925a2e2bea8c3ffaf01eb824ad66716
df5389da6e29392ef512f78021cfe3e973a9f67432db8e3ebb3c08884fea6b49
ea55637605369034ac4790dfb67935ad71a8a96849ea7450979f2f0eb2e628c8
2884b1be88a167c3488824ca6c6d2318971b0ad0b10cd2e89b6cda7c3e59fa3a
72f3459f8ae47beb0054c8dd4ef4db69a7d14b479c322cd97fdbd17da1664d84
778393a87d0f30fdb6cd8ff6b1ab70d5d37f3ac25ca3c6b1bea5246b5b0d23b4
83722f3d03eea8f67ec46876f9f0055b1b91c68ec832d8ddaaa35e408160bd55
84ca7279b037d51e4634a8bccd8fd218328ab23ec174d4a7695b3f4d2e667592
e4514aaff4ca0e4dd9fded6066783692417a435fc35bf0ed37ddc9252af35bf6
5d3fa9cbe7dcfab4944c90e396c6b50743127e88f5ecb09e65c32e74c89393f1
5e3718c3f6d86bf10ce51fe0eb2f71f639036d6087c897e029cf58863bd5374a
6156f40ae32eb946f0f8e857ba95ed2b50ff9a106d46bcb8e1fa70208e9d28b5
90b9247577d572c10639ae0f6118d5e6501c787d45b78b3754c07328003e81ed
bdcfc3a90c5b4b1f74c9387fb599a0076e9caee0cb5dfdb1664f22f9825446ed
018bdaf1fc011d7c36ff885c6428439935c93eefa365d3a3e6c35354ed3d1ea6
565a0231

In [ ]:
#Purge all docker networks
!docker network prune -f